# Final Evaluation: Test Set Performance

================================================================================
PURPOSE: Evaluate final models on the held-out test set
================================================================================

**CRITICAL**: This notebook evaluates on the **TEST set** which was separated
at the beginning (in 01_data_split.ipynb) and has NEVER been used for training,
model selection, feature selection, or any development decisions.

**Evaluation Protocol:**
1. Load best model/classifier combinations (based on Dev set results from
   03_train_evaluate.ipynb and 04_early_fusion.ipynb)
2. Extract features for TEST set (if not already extracted)
3. Retrain models on combined Train+Dev data (final training)
4. Evaluate on TEST set (unbiased final performance estimate)
5. Generate comprehensive final reports, plots, and summary tables

**Important Notes:**
- Test set is ONLY accessed in this notebook
- Models are retrained on Train+Dev before test evaluation
- This provides an unbiased estimate of generalization performance
- Results from this notebook represent the final competition submission metrics

================================================================================
INPUTS (What this notebook loads)
================================================================================

**From GitHub:**
- Repository code (cloned automatically if not present)
- Source modules from `src/` directory:
  - `src.storage.manager` (StorageManager)
  - `src.features.extraction` (feature extraction functions)
  - `src.models.trainer` (training and evaluation functions)
  - `src.models.classifiers` (classifier definitions)
  - `src.evaluation.metrics` (metric computation functions)
  - `src.evaluation.tables` (results table functions)
  - `src.evaluation.visualizer` (plotting functions)

**From Google Drive:**
- Dataset splits: `splits/dataset_splits.pkl`
  - Test split (ONLY accessed in this notebook, 308 samples)
  - Train and Dev splits (for combined training)
- Feature matrices (if already extracted):
  - `features/raw/X_test_{model}_{task}.npy` (may not exist yet)
  - `features/raw/X_train_{model}_{task}.npy` (from 02_feature_extraction_separate.ipynb)
  - `features/raw/X_dev_{model}_{task}.npy` (from 02_feature_extraction_separate.ipynb)

**From HuggingFace Hub:**
- Transformer models (if test features need extraction):
  - BERT, RoBERTa, DeBERTa, XLNet tokenizers and models

================================================================================
OUTPUTS (What this notebook saves)
================================================================================

**To Google Drive:**
- Test features (if extracted): `features/raw/X_test_{model}_{task}.npy`
  - For each model (bert, roberta, deberta, xlnet)
  - For each task (clarity, evasion)
  - Shape: (308_samples, 19_features)
- Final predictions: `predictions/pred_test_{model}_{classifier}_{task}.npy`
  - Hard label predictions for Test set
  - For each model/classifier/task combination
- Final probabilities: `features/probabilities/probs_test_{model}_{classifier}_{task}.npy`
  - Probability distributions for Test set
  - For each model/classifier/task combination
- Final evaluation plots: `plots/final_evaluation/{model}_{task}_{classifier}/`
  - Confusion matrices
  - Precision-Recall curves
  - ROC curves

**To GitHub:**
- Test feature metadata: `metadata/features_test_{model}_{task}.json`
  - Feature names and dimensions
  - Timestamp information
- Final results metadata: `results/FINAL_TEST_{model}_{task}.json`
  - Final test set metrics
  - Model/classifier/task information
  - Test sample counts

**Evaluation Metrics Computed and Printed:**
- Accuracy
- Macro Precision, Recall, F1
- Weighted Precision, Recall, F1
- Per-class metrics (precision, recall, F1, support)
- Cohen's Kappa
- Matthews Correlation Coefficient
- Hamming Loss
- Jaccard Score (IoU)
- Confusion Matrix

**What represents final competition submission:**
- All test set predictions and probabilities
- Final evaluation metrics (computed on 308 test samples)
- Complete evaluation results for all model/classifier/task combinations


In [ ]:
# ============================================================================
# SETUP: Repository Clone, Drive Mount, and Path Configuration
# ============================================================================
# This cell performs minimal setup required for the notebook to run:
# 1. Clones repository from GitHub (if not already present)
# 2. Mounts Google Drive for persistent data storage
# 3. Configures Python paths and initializes StorageManager
# 4. Loads test split (ONLY accessed in this notebook)

import shutil
import os
import subprocess
import time
import requests
import zipfile
import sys
from pathlib import Path
from google.colab import drive
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

# Repository configuration
repo_dir = '/content/semeval-context-tree-modular'
repo_url = 'https://github.com/EonTechie/semeval-context-tree-modular.git'
zip_url = 'https://github.com/EonTechie/semeval-context-tree-modular/archive/refs/heads/main.zip'

# Clone repository (if not already present)
if not os.path.exists(repo_dir):
    print("Cloning repository from GitHub...")
    max_retries = 2
    clone_success = False
    
    for attempt in range(max_retries):
        try:
            result = subprocess.run(
                ['git', 'clone', repo_url],
                cwd='/content',
                capture_output=True,
                text=True,
                timeout=60
            )
            if result.returncode == 0:
                print("Repository cloned successfully via git")
                clone_success = True
                break
            else:
                if attempt < max_retries - 1:
                    time.sleep(3)
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(3)
    
    # Fallback: Download as ZIP if git clone fails
    if not clone_success:
        print("Git clone failed. Downloading repository as ZIP archive...")
        zip_path = '/tmp/repo.zip'
        try:
            response = requests.get(zip_url, stream=True, timeout=60)
            response.raise_for_status()
            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall('/content')
            extracted_dir = '/content/semeval-context-tree-modular-main'
            if os.path.exists(extracted_dir):
                os.rename(extracted_dir, repo_dir)
            os.remove(zip_path)
            print("Repository downloaded and extracted successfully")
        except Exception as e:
            raise RuntimeError(f"Failed to obtain repository: {e}")

# Mount Google Drive (if not already mounted)
try:
    drive.mount('/content/drive', force_remount=False)
except Exception:
    pass  # Already mounted

# Configure paths
BASE_PATH = Path('/content/semeval-context-tree-modular')
DATA_PATH = Path('/content/drive/MyDrive/semeval_data')

# Verify repository structure exists
if not BASE_PATH.exists():
    raise RuntimeError(f"Repository directory not found: {BASE_PATH}")
if not (BASE_PATH / 'src').exists():
    raise RuntimeError(f"src directory not found in repository: {BASE_PATH / 'src'}")
if not (BASE_PATH / 'src' / 'storage' / 'manager.py').exists():
    raise RuntimeError(f"Required file not found: {BASE_PATH / 'src' / 'storage' / 'manager.py'}")

# Add repository to Python path
sys.path.insert(0, str(BASE_PATH))

# Verify imports work
try:
    from src.storage.manager import StorageManager
    from src.features.extraction import featurize_hf_dataset_in_batches_v2
    from src.models.classifiers import get_classifier_dict
    from src.evaluation.metrics import compute_all_metrics, print_classification_report
    from src.evaluation.tables import print_results_table
    from src.evaluation.visualizer import visualize_all_evaluation
except ImportError as e:
    raise ImportError(
        f"Failed to import required modules. "
        f"Repository path: {BASE_PATH}, "
        f"Python path: {sys.path[:3]}, "
        f"Error: {e}"
    )

# Initialize StorageManager
storage = StorageManager(
    base_path=str(BASE_PATH),
    data_path=str(DATA_PATH),
    github_path=str(BASE_PATH)
)

# Test splits will be loaded per-task in the evaluation loop
# Clarity and Evasion have different test splits (Evasion uses majority voting)

print("Setup complete")
print(f"  Repository: {BASE_PATH}")
print(f"  Data storage: {DATA_PATH}")
print(f"\nCRITICAL: Test sets will be loaded per-task (task-specific splits)")
print("         Clarity and Evasion have different test splits due to majority voting")
print("         These sets have NEVER been used for training or development!")


In [ ]:
# ============================================================================
# CONFIGURE MODELS, TASKS, AND CLASSIFIERS
# ============================================================================
# Defines the models, tasks, and classifiers for final evaluation
# NOTE: In practice, you should select best model/classifier based on Dev set
# results. For comprehensive comparison, all combinations are evaluated here.

MODELS = ['bert', 'bert_political', 'bert_ambiguity', 'roberta', 'deberta', 'xlnet']
TASKS = ['clarity', 'evasion']

# Label mappings for each task
CLARITY_LABELS = ['Clear Reply', 'Ambiguous', 'Clear Non-Reply']
EVASION_LABELS = ['Direct Answer', 'Partial Answer', 'Implicit Answer', 
                  'Uncertainty', 'Refusal', 'Clarification', 
                  'Question', 'Topic Shift', 'Other']

# Initialize classifiers with fixed random seed for reproducibility
classifiers = get_classifier_dict(random_state=42)

# Configure device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Configuration for final evaluation:")
print(f"  Models: {MODELS}")
print(f"  Tasks: {TASKS}")
print(f"  Classifiers: {list(classifiers.keys())}")
print(f"  Device: {device}")
print(f"\nNOTE: Evaluating all model/classifier combinations on TEST set")
print("      In practice, select best combination based on Dev set results")


In [ ]:
# ============================================================================
# EXTRACT TEST SET FEATURES (IF NOT ALREADY EXTRACTED)
# ============================================================================
# Checks if test features already exist, otherwise extracts them
# Test features are extracted separately to ensure test set is only accessed
# in this final evaluation notebook

MODEL_CONFIGS = {
    'bert': 'bert-base-uncased',
    'bert_political': 'bert-base-uncased',  # TODO: Replace with actual political discourse BERT model
    'bert_ambiguity': 'bert-base-uncased',  # TODO: Replace with actual ambiguity-focused BERT model
    'roberta': 'roberta-base',
    'deberta': 'microsoft/deberta-v3-base',
    'xlnet': 'xlnet-base-cased'
}

for model_key, model_name in MODEL_CONFIGS.items():
    print(f"\n{'='*80}")
    print(f"Processing {model_key.upper()}")
    print(f"{'='*80}")
    
    # Load transformer model and tokenizer from HuggingFace Hub
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    model.to(device)
    model.eval()
    
    # Get model-specific max sequence length from tokenizer or model config
    # Use tokenizer.model_max_length if available, otherwise use model.config.max_position_embeddings
    if hasattr(tokenizer, 'model_max_length') and tokenizer.model_max_length is not None and tokenizer.model_max_length < 1e10:
        max_seq_len = tokenizer.model_max_length
    elif hasattr(model.config, 'max_position_embeddings'):
        max_seq_len = model.config.max_position_embeddings
    else:
        # Fallback: use default based on model type
        if 'xlnet' in model_name.lower():
            max_seq_len = 1024  # XLNet typically supports 1024
        else:
            max_seq_len = 512   # BERT, RoBERTa, DeBERTa typically 512
    
    print(f"Max sequence length for {model_key}: {max_seq_len}")
    
    for task in TASKS:
        print(f"\n{'='*60}")
        print(f"Task: {task.upper()}")
        print(f"{'='*60}")
        
        # Check if TEST features already exist in persistent storage
        try:
            X_test = storage.load_features(model_key, task, 'test')
            print(f"Test features already exist: {model_key}_{task}")
        except FileNotFoundError:
            print(f"Test features not found for {model_key}_{task}, extracting...")
            
            # Load task-specific train split to fit TF-IDF vectorizer for this task
            # Each task gets its own TF-IDF vectorizer (as in siparismaili01)
            train_ds_temp = storage.load_split('train', task=task)
            
            # Fit TF-IDF on train split for this task
            print(f"  Fitting TF-IDF vectorizer on train split for {task}...")
            _, _, tfidf_vectorizer = featurize_hf_dataset_in_batches_v2(
                train_ds_temp,
                tokenizer,
                model,
                device,
                batch_size=8,
                max_sequence_length=max_seq_len,  # Model-specific max sequence length
                question_key='interview_question',  # Key for question text in dataset (original question, NOT 'question' which is paraphrased)
                answer_key='interview_answer',  # Key for answer text in dataset (QEvasion uses 'interview_answer')
                show_progress=False,  # No progress bar for TF-IDF fitting
                tfidf_vectorizer=None  # Fit new TF-IDF on train for this task
            )
            
            # Load task-specific test split
            test_ds = storage.load_split('test', task=task)
            
            # Extract 19 Context Tree features for test set using TF-IDF fitted on train
            print(f"  Extracting test features using TF-IDF fitted on train for {task}...")
            X_test, feature_names, _ = featurize_hf_dataset_in_batches_v2(
                test_ds,
                tokenizer,
                model,
                device,
                batch_size=8,              # Batch size for feature extraction
                max_sequence_length=max_seq_len,  # Model-specific max sequence length
                question_key='interview_question',  # Key for question text in dataset (original question, NOT 'question' which is paraphrased)
                answer_key='interview_answer',  # Key for answer text in dataset (QEvasion uses 'interview_answer')
                show_progress=True,         # Show progress bar
                tfidf_vectorizer=tfidf_vectorizer  # Reuse TF-IDF from train (no leakage)
            )
            
            # Save test features to persistent storage
            storage.save_features(
                X_test, model_key, task, 'test', feature_names
            )
            
            print(f"  Saved: {X_test.shape[0]} samples, {X_test.shape[1]} features")
    
    # Free GPU memory after processing each model
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("\nTest feature extraction complete")
print("  All test features are now available for final evaluation")


In [ ]:
# ============================================================================
# HIERARCHICAL EVALUATION ON TEST SET: EVASION PREDICTIONS → CLARITY PREDICTIONS
# ============================================================================
# For each model, uses evasion TEST predictions to generate clarity predictions
# via hierarchical mapping, then evaluates against true TEST clarity labels
# This is the final evaluation of the hierarchical approach on the test set

from src.models.hierarchical import evaluate_hierarchical_approach
from sklearn.preprocessing import LabelEncoder
import numpy as np

hierarchical_final_results = {}

for model in MODELS:
    print(f"\n{'='*80}")
    print(f"MODEL: {model.upper()} - HIERARCHICAL EVASION → CLARITY (TEST SET)")
    print(f"{'='*80}")
    
    # Check if we have evasion test predictions for this model
    if 'evasion' not in final_results.get(model, {}):
        print(f"  Skipping {model}: No evasion test predictions available")
        continue
    
    # Get evasion test results
    evasion_test_results = final_results[model]['evasion']
    
    # Use predictions from best classifier (by Macro F1) or MLP if available
    best_classifier = None
    best_f1 = -1
    
    for clf_name, clf_result in evasion_test_results.items():
        if 'metrics' in clf_result:
            f1 = clf_result['metrics'].get('macro_f1', 0.0)
            if f1 > best_f1:
                best_f1 = f1
                best_classifier = clf_name
    
    if best_classifier is None:
        print(f"  Skipping {model}: No valid evasion test predictions found")
        continue
    
    print(f"  Using evasion test predictions from: {best_classifier} (Macro F1: {best_f1:.4f})")
    
    # Load test set for evasion task (275 samples after majority voting)
    test_ds_evasion = storage.load_split('test', task='evasion')
    
    # Get evasion test predictions (string labels from classifier)
    # These are the predictions from the best classifier on test set
    y_evasion_pred_test_strings = evasion_test_results[best_classifier]['predictions']
    
    # Get true evasion labels from test set
    y_evasion_true_test = np.array([test_ds_evasion[i]['evasion_label'] for i in range(len(test_ds_evasion))])
    
    # Get true clarity labels from test set (same 275 samples)
    y_clarity_true_test = np.array([test_ds_evasion[i]['clarity_label'] for i in range(len(test_ds_evasion))])
    
    # Encode labels for evaluation (both true and predicted)
    le_evasion = LabelEncoder()
    le_clarity = LabelEncoder()
    
    # Fit encoders on true labels to ensure consistent encoding
    y_evasion_true_encoded = le_evasion.fit_transform(y_evasion_true_test)
    y_clarity_true_encoded = le_clarity.fit_transform(y_clarity_true_test)
    
    # Encode evasion predictions using the same encoder
    # This ensures predictions are in the same encoding space as true labels
    y_evasion_pred_test = le_evasion.transform(y_evasion_pred_test_strings)
    
    # Evaluate hierarchical approach on TEST set
    hierarchical_metrics = evaluate_hierarchical_approach(
        y_evasion_true_encoded,
        y_evasion_pred_test,
        y_clarity_true_encoded,
        EVASION_LABELS,
        CLARITY_LABELS
    )
    
    hierarchical_final_results[model] = {
        'classifier': best_classifier,
        'metrics': hierarchical_metrics,
        'evasion_f1': best_f1,
        'n_test': len(y_clarity_true_test)
    }
    
    print(f"\n  Hierarchical Clarity Performance (TEST SET):")
    print(f"    Test samples: {len(y_clarity_true_test)}")
    print(f"    Accuracy: {hierarchical_metrics['accuracy']:.4f}")
    print(f"    Macro F1: {hierarchical_metrics['macro_f1']:.4f}")
    print(f"    Weighted F1: {hierarchical_metrics['weighted_f1']:.4f}")
    
    # Print detailed classification report
    print_classification_report(
        y_clarity_true_encoded,
        hierarchical_metrics['predictions'],
        CLARITY_LABELS,
        task_name=f"TEST - {model} - Hierarchical Evasion→Clarity"
    )
    
    # Add to final_results for summary
    if 'hierarchical_evasion_to_clarity' not in final_results[model]:
        final_results[model]['hierarchical_evasion_to_clarity'] = {}
    
    final_results[model]['hierarchical_evasion_to_clarity'][best_classifier] = {
        'metrics': hierarchical_metrics,
        'predictions': hierarchical_metrics['predictions'],
        'probabilities': None  # Hierarchical approach doesn't produce probabilities
    }
    
    # Save hierarchical test predictions
    storage.save_predictions(
        hierarchical_metrics['predictions'],
        model, best_classifier, 'hierarchical_evasion_to_clarity', 'test'
    )
    
    # Save hierarchical results to metadata
    experiment_id = f"FINAL_TEST_{model}_hierarchical_evasion_to_clarity"
    storage.save_results({
        'split': 'test',
        'model': model,
        'task': 'hierarchical_evasion_to_clarity',
        'n_test': len(y_clarity_true_test),
        'evasion_classifier': best_classifier,
        'evasion_f1': best_f1,
        'results': {
            best_classifier: {'metrics': hierarchical_metrics}
        }
    }, experiment_id)

print(f"\n{'='*80}")
print("HIERARCHICAL EVALUATION ON TEST SET COMPLETE")
print(f"{'='*80}")
print("\nSummary:")
print("  - Evasion test predictions mapped to clarity predictions")
print("  - Hierarchical approach evaluated on TEST set (275 samples)")
print("  - Test set clarity labels used as gold standard")
print("  - Results added to final_results for final summary")


In [ ]:
# ============================================================================
# FINAL EVALUATION ON TEST SET
# ============================================================================
# Retrains models on combined Train+Dev data and evaluates on Test set
# This provides an unbiased estimate of generalization performance
# Results represent the final competition submission metrics

final_results = {}

for model in MODELS:
    print(f"\n{'='*80}")
    print(f"MODEL: {model.upper()} - FINAL EVALUATION ON TEST SET")
    print(f"{'='*80}")
    
    final_results[model] = {}
    
    for task in TASKS:
        print(f"\n{'='*60}")
        print(f"TASK: {task.upper()}")
        print(f"{'='*60}")
        
        # Select appropriate label list and dataset key for this task
        if task == 'clarity':
            label_list = CLARITY_LABELS
            label_key = 'clarity_label'
        else:  # evasion
            label_list = EVASION_LABELS
            label_key = 'evasion_label'
        
        # Load task-specific splits (Clarity and Evasion have different splits)
        # Evasion splits are filtered by majority voting
        test_ds = storage.load_split('test', task=task)
        train_ds = storage.load_split('train', task=task)
        dev_ds = storage.load_split('dev', task=task)
        
        print(f"  Loaded splits for {task}:")
        print(f"    Train: {len(train_ds)} samples")
        print(f"    Dev: {len(dev_ds)} samples")
        print(f"    Test: {len(test_ds)} samples")
        
        # Extract test labels
        y_test = np.array([test_ds[i][label_key] for i in range(len(test_ds))])
        
        # Load test features
        X_test = storage.load_features(model, task, 'test')
        
        # Load train features and labels for final training
        X_train = storage.load_features(model, task, 'train')
        y_train = np.array([train_ds[i][label_key] for i in range(len(train_ds))])
        
        # Load dev features and labels (will be combined with train)
        X_dev = storage.load_features(model, task, 'dev')
        y_dev = np.array([dev_ds[i][label_key] for i in range(len(dev_ds))])
        
        # Combine train + dev for final training
        # This maximizes training data before final evaluation
        X_train_full = np.vstack([X_train, X_dev])
        y_train_full = np.concatenate([y_train, y_dev])
        
        print(f"  Training on: {X_train_full.shape[0]} samples (train+dev combined)")
        print(f"  Testing on: {X_test.shape[0]} samples (test set)")
        
        # Train all classifiers on full train+dev and evaluate on test
        task_results = {}
        
        for classifier_name, clf in classifiers.items():
            print(f"\n  Training {classifier_name} on full train+dev...")
            
            # Train on combined train+dev data
            clf.fit(X_train_full, y_train_full)
            
            # Predict on test set
            y_test_pred = clf.predict(X_test)
            
            # Get probability distributions (if classifier supports it)
            try:
                y_test_proba = clf.predict_proba(X_test)
            except AttributeError:
                y_test_proba = None
            
            # Compute comprehensive evaluation metrics
            metrics = compute_all_metrics(
                y_test, y_test_pred, label_list, 
                task_name=f"TEST_{model}_{task}_{classifier_name}"
            )
            
            # Print detailed classification report
            print_classification_report(
                y_test, y_test_pred, label_list,
                task_name=f"TEST - {model} - {task} - {classifier_name}"
            )
            
            # Generate evaluation plots (confusion matrix, PR curves, ROC curves)
            if y_test_proba is not None:
                visualize_all_evaluation(
                    y_test, y_test_pred, y_test_proba, label_list,
                    task_name=f"TEST_{model}_{task}",
                    classifier_name=classifier_name,
                    save_dir=str(DATA_PATH / 'plots' / 'final_evaluation')
                )
            
            task_results[classifier_name] = {
                'metrics': metrics,
                'predictions': y_test_pred,
                'probabilities': y_test_proba
            }
            
            # Save test predictions and probabilities to persistent storage
            storage.save_predictions(
                y_test_pred, model, classifier_name, task, 'test'
            )
            if y_test_proba is not None:
                storage.save_probabilities(
                    y_test_proba, model, classifier_name, task, 'test'
                )
        
        # Print results comparison table for this model/task
        print_results_table(
            {name: {'metrics': res['metrics']} for name, res in task_results.items()},
            task_name=f"TEST - {model} - {task}",
            sort_by="Macro F1"
        )
        
        final_results[model][task] = task_results
        
        # Save final results to metadata
        experiment_id = f"FINAL_TEST_{model}_{task}"
        storage.save_results({
            'split': 'test',
            'model': model,
            'task': task,
            'n_test': len(y_test),
            'results': {
                name: {'metrics': res['metrics']}
                for name, res in task_results.items()
            }
        }, experiment_id)

print(f"\n{'='*80}")
print("FINAL EVALUATION ON TEST SET COMPLETE")
print(f"{'='*80}")
print("\nSummary:")
print("  - All models retrained on combined Train+Dev data")
print("  - Final evaluation performed on held-out Test set")
print("  - Test predictions and probabilities saved to Google Drive")
print("  - Final results tables and plots generated")
print("  - Results represent final competition submission metrics")
```

Cell 5:
```
# ============================================================================
# HIERARCHICAL EVALUATION ON TEST SET: EVASION PREDICTIONS → CLARITY PREDICTIONS
# ============================================================================
# For each model, uses evasion TEST predictions to generate clarity predictions
# via hierarchical mapping, then evaluates against true TEST clarity labels
# This is the final evaluation of the hierarchical approach on the test set

from src.models.hierarchical import evaluate_hierarchical_approach
from sklearn.preprocessing import LabelEncoder
import numpy as np

hierarchical_final_results = {}

for model in MODELS:
    print(f"\n{'='*80}")
    print(f"MODEL: {model.upper()} - HIERARCHICAL EVASION → CLARITY (TEST SET)")
    print(f"{'='*80}")
    
    # Check if we have evasion test predictions for this model
    if 'evasion' not in final_results.get(model, {}):
        print(f"  Skipping {model}: No evasion test predictions available")
        continue
    
    # Get evasion test results
    evasion_test_results = final_results[model]['evasion']
    
    # Use predictions from best classifier (by Macro F1) or MLP if available
    best_classifier = None
    best_f1 = -1
    
    for clf_name, clf_result in evasion_test_results.items():
        if 'metrics' in clf_result:
            f1 = clf_result['metrics'].get('macro_f1', 0.0)
            if f1 > best_f1:
                best_f1 = f1
                best_classifier = clf_name
    
    if best_classifier is None:
        print(f"  Skipping {model}: No valid evasion test predictions found")
        continue
    
    print(f"  Using evasion test predictions from: {best_classifier} (Macro F1: {best_f1:.4f})")
    
    # Load test set for evasion task (275 samples after majority voting)
    test_ds_evasion = storage.load_split('test', task='evasion')
    
    # Get evasion test predictions (encoded as integers)
    # These are the predictions from the best classifier on test set
    y_evasion_pred_test = evasion_test_results[best_classifier]['predictions']
    
    # Get true evasion labels from test set
    y_evasion_true_test = np.array([test_ds_evasion[i]['evasion_label'] for i in range(len(test_ds_evasion))])
    
    # Get true clarity labels from test set (same 275 samples)
    y_clarity_true_test = np.array([test_ds_evasion[i]['clarity_label'] for i in range(len(test_ds_evasion))])
    
    # Encode labels for evaluation
    le_evasion = LabelEncoder()
    le_clarity = LabelEncoder()
    
    y_evasion_true_encoded = le_evasion.fit_transform(y_evasion_true_test)
    y_clarity_true_encoded = le_clarity.fit_transform(y_clarity_true_test)
    
    # Evaluate hierarchical approach on TEST set
    hierarchical_metrics = evaluate_hierarchical_approach(
        y_evasion_true_encoded,
        y_evasion_pred_test,
        y_clarity_true_encoded,
        EVASION_LABELS,
        CLARITY_LABELS
    )
    
    hierarchical_final_results[model] = {
        'classifier': best_classifier,
        'metrics': hierarchical_metrics,
        'evasion_f1': best_f1,
        'n_test': len(y_clarity_true_test)
    }
    
    print(f"\n  Hierarchical Clarity Performance (TEST SET):")
    print(f"    Test samples: {len(y_clarity_true_test)}")
    print(f"    Accuracy: {hierarchical_metrics['accuracy']:.4f}")
    print(f"    Macro F1: {hierarchical_metrics['macro_f1']:.4f}")
    print(f"    Weighted F1: {hierarchical_metrics['weighted_f1']:.4f}")
    
    # Print detailed classification report
    print_classification_report(
        y_clarity_true_encoded,
        hierarchical_metrics['predictions'],
        CLARITY_LABELS,
        task_name=f"TEST - {model} - Hierarchical Evasion→Clarity"
    )
    
    # Add to final_results for summary
    if 'hierarchical_evasion_to_clarity' not in final_results[model]:
        final_results[model]['hierarchical_evasion_to_clarity'] = {}
    
    final_results[model]['hierarchical_evasion_to_clarity'][best_classifier] = {
        'metrics': hierarchical_metrics,
        'predictions': hierarchical_metrics['predictions'],
        'probabilities': None  # Hierarchical approach doesn't produce probabilities
    }
    
    # Save hierarchical test predictions
    storage.save_predictions(
        hierarchical_metrics['predictions'],
        model, best_classifier, 'hierarchical_evasion_to_clarity', 'test'
    )
    
    # Save hierarchical results to metadata
    experiment_id = f"FINAL_TEST_{model}_hierarchical_evasion_to_clarity"
    storage.save_results({
        'split': 'test',
        'model': model,
        'task': 'hierarchical_evasion_to_clarity',
        'n_test': len(y_clarity_true_test),
        'evasion_classifier': best_classifier,
        'evasion_f1': best_f1,
        'results': {
            best_classifier: {'metrics': hierarchical_metrics}
        }
    }, experiment_id)

print(f"\n{'='*80}")
print("HIERARCHICAL EVALUATION ON TEST SET COMPLETE")
print(f"{'='*80}")
print("\nSummary:")
print("  - Evasion test predictions mapped to clarity predictions")
print("  - Hierarchical approach evaluated on TEST set (275 samples)")
print("  - Test set clarity labels used as gold standard")
print("  - Results added to final_results for final summary")
